<a href="https://colab.research.google.com/github/TeneikaAskew/automating-web-scraping-text-calendar-notifications/blob/main/1_1_Webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose of the Script
This script is designed to scrape session data from the **Techsgiving Summit** website and then store it into a Google Sheet. The script performs the following tasks:

1. **Scrape HTML**: The HTML content of the summit's schedule page is fetched and processed.
2. **Extract Session Data**: Key information such as session title, start time, end time, track, description, speakers, and location are extracted from the HTML.
3. **Store Data in Google Sheets**: The extracted session data is stored in a Google Sheet for further use, with added data validation for the "Planning to Go?" column.
4. **Log Extracted Data**: The data is logged to the console for debugging purposes.

**Developer**: Dakota Smith



In [ ]:
# @title
%%javascript

function parseHTMLtoData(html) {
  const url = "https://www.techsgivingsummit.com/schedule"; // Replace with your actual URL

  // Fetch the HTML content
  const response = UrlFetchApp.fetch(url);
  html = response.getContentText();//.replace(/ w--tab-active/g, '');//.replace(' w--tab-active','');
  // console.log(html.substring(13000, 13061))
  // console.log(html.substring(24759, 25759))

  // Find all occurrences of '<div class="w-tab-content'
  const indices = [];
  let startIndex = html.indexOf('w-tab-pane');
  while (startIndex !== -1) {
    indices.push(startIndex-75);
    startIndex = html.indexOf('w-tab-pane', startIndex + 1);
  }

  console.log("Indices of '<div class=\"w-tab-pane':", indices);

  // Split the HTML into sections based on the indices
  const sections = [];
  for (let i = 0; i < indices.length; i++) {
    const start = indices[i];
    const end = i < indices.length - 1 ? indices[i + 1] : html.length; // Use the next index or the end of the HTML
    sections.push(html.substring(start, end));
    // console.log(`Section ${i + 1}:`, sections);
  }
  console.log(`Total Sections Found: ${sections.length}`);

  // Process each section to extract tabs and sessions
  const sessions = [];
  sections.forEach((section, index) => {
    // Extract tab name
    //  console.log(section.substring(0, 150));
    // const dateMatch = section.match(/data-w-tab="([^"]+)"/);
    // Look for the first data-w-tab in the section
    const dateMatch = section.match(/data-w-tab="([^"]+)"/);
    if (dateMatch) {
      console.log(`Section ${index + 1} Matched Date:`, dateMatch[1]);
      // console.log(`Section Content Preview:`, section.substring(0, 500)); // Log the surrounding context
    } else {
      console.log(`No date found in Section ${index + 1}`);
    }
    const currentDate = dateMatch ? dateMatch[1].replace("th", ", 2024") : "Unknown";
    // console.log(`Processing Section ${index + 1}: Tab Name - ${currentDate}`);

    // Regex to extract tabs with their `data-w-tab` attribute and sessions
    const sessionRegex = /<div class="speaker-session-wrapper">([\s\S]*?)(?=<div class="speaker-session-wrapper"|<\/div><\/div><\/div><\/div>)/g;
    const sessionMatches = section.match(sessionRegex);

    if (sessionMatches) {
      sessionMatches.forEach(sessionHtml => {
        // Field regexes
        const timeStartRegex = /<div class="text-block-230">(.*?)<\/div>/;
        const timeEndRegex = /<div class="text-block-230 end">(.*?)<\/div>/;
        const trackRegex = /<div[^>]*class="session-track"[^>]*>(.*?)<\/div>/;
        const titleRegex = /<h1 class="session-ttitle">(.*?)<\/h1>/;
        const descriptionRegex = /<p class="paragraph-8">(.*?)<\/p>/;
        const speakerRegex = /<div class="session-speaker">(.*?)<\/div>/g;
        const locationRegex = /<div class="session-location">\s*<a[^>]*>(.*?)<\/a>/;

        // Extract individual fields
        const timeStartRaw = extractField(sessionHtml, timeStartRegex);
        const timeStart = timeStartRaw ? timeStartRaw.replace('-', '') : null;
        const timeEnd = extractField(sessionHtml, timeEndRegex);
        const track = extractField(sessionHtml, trackRegex);
        const title = extractField(sessionHtml, titleRegex);
        const description = extractField(sessionHtml, descriptionRegex);

      // Extract multiple speakers
      const speakers = [];
      let speakerMatch;
      while ((speakerMatch = speakerRegex.exec(sessionHtml.replace('session-speaker name', 'session-speaker'))) !== null) {
        speakers.push(speakerMatch[1].trim());
        console.log(speakers)
      }
      const speakersString = speakers.join(", "); // Join speakers with a comma and space
      console.log(speakersString)

      const location = extractField(sessionHtml, locationRegex).replace('<div class="text-block-231">','').replace('</div>','');

      // Only include sessions with at least time_start and title
      if (timeStart || title) {
        sessions.push({
          date: currentDate, // Add the event day
          time_start: timeStart,
          time_end: timeEnd,
          track,
          title,
          speakers: speakersString,
          description,
          location,
          });
        }
      });
    } else {
      console.log(`No sessions found for ${tabName}`);
    }
  });
  // Log the extracted data
  Logger.log(JSON.stringify(sessions, null, 2));

  // Step 5: Write data to Google Sheets
  const sheetName = "AppScript_Sessions";
  let sheet = SpreadsheetApp.getActiveSpreadsheet().getSheetByName(sheetName);
  if (!sheet) {
    sheet = SpreadsheetApp.getActiveSpreadsheet().insertSheet(sheetName);
  }
  sheet.clear();
  sheet.appendRow(["Date","Start Time", "End Time", "Track", "Title", "Speakers", "Description", "Location", "Planning to Go?"]);
  sessions.forEach(session => {
    sheet.appendRow([
      session.date,
      session.time_start,
      session.time_end,
      session.track,
      session.title,
      session.speakers,
      session.description,
      session.location,
    ]);
  });

  // Add data validation to "Planning to Go?" column
  const validationRange = sheet.getRange(2, 9, sheet.getLastRow() - 1); // "Planning to Go?" column (I)
  const rule = SpreadsheetApp.newDataValidation()
    .requireValueInList(["Yes", "No"], true)
    .setAllowInvalid(false)
    .build();
  validationRange.setDataValidation(rule);
  return sessions;
}

function extractField(html, regex) {
  const match = html.match(regex);
  return match ? match[1].trim() : null;
}

function testRegexInAppScript() {
  const htmlSnippet = '<div class="w-tab-content"><div class="w-tab-pane w--tab-active" data-w-tab="December 11th" id="Tab-1"><div class="december-11-wrapper"><div class="collection-list-wrapper-2 w-dyn-list"><div class="w-tab-pane" data-w-tab="December 12th"><div class="december-12th-wrapper" id="D">';

  // Regex to match the data-w-tab attribute
  const tabNameRegex = /data-w-tab="([^"]+)"/;

  // Extract the match
  const match = htmlSnippet.match(tabNameRegex);
  if (match) {
    Logger.log(`Extracted tab name: ${match[1]}`);
  } else {
    Logger.log("No match found.");
  }
}




### Breakdown of the Function:


1. **Fetching HTML Content:**
   - The function begins by fetching the HTML content of the summit schedule page using `UrlFetchApp.fetch()`.
   - The HTML content is then retrieved using `.getContentText()` and stored in the variable `html`.


In [ ]:
%%javascript
function parseHTMLtoData(html) {
  const url = "https://www.techsgivingsummit.com/schedule"; // Replace with your actual URL

  // Fetch the HTML content
  const response = UrlFetchApp.fetch(url);
  html = response.getContentText();//.replace(/ w--tab-active/g, '');//.replace(' w--tab-active','');
  // console.log(html.substring(13000, 13061))
  // console.log(html.substring(24759, 25759))

2. **Identifying Section Indices:**
   - The function looks for all occurrences of the `<div class="w-tab-pane"` HTML tag, which is used to mark different sections of the page.
   - The `indexOf()` method is used to find each occurrence, and the starting indices are saved in the `indices` array. This array will help split the HTML into sections later on.


In [ ]:
%%javascript
  // Find all occurrences of '<div class="w-tab-content'
  const indices = [];
  let startIndex = html.indexOf('w-tab-pane');
  while (startIndex !== -1) {
    indices.push(startIndex-75);
    startIndex = html.indexOf('w-tab-pane', startIndex + 1);
  }

  console.log("Indices of '<div class=\"w-tab-pane':", indices);

3. **Splitting HTML Into Sections:**
   - The HTML content is split into individual sections using the indices found in the previous step. Each section corresponds to a distinct tab of the summit schedule (e.g., different dates).
   - The HTML for each section is extracted using `substring()` and stored in the `sections` array.

In [ ]:
%%javascript
  // Process each section to extract tabs and sessions
  const sessions = [];

4. **Processing Each Section:**
   - The function loops through each section and extracts the tab name (which corresponds to a specific date).
   - The tab name is retrieved using a regular expression (`data-w-tab="([^"]+)"`), which matches the `data-w-tab` attribute that holds the date information.


In [ ]:
%%javascript
  sections.forEach((section, index) => {
    // Extract tab name
    //  console.log(section.substring(0, 150));
    // const dateMatch = section.match(/data-w-tab="([^"]+)"/);
    // Look for the first data-w-tab in the section

5. **Extracting Session Details:**
   - For each section, the function uses regular expressions to extract session details:
     - **Time**: Extracted using the regex `/<div class="text-block-230">(.*?)<\/div>/`.
     - **Track**: Extracted using the regex `/<div[^>]*class="session-track"[^>]*>(.*?)<\/div>/`.
     - **Title**: Extracted using the regex `/<h1 class="session-ttitle">(.*?)<\/h1>/`.
     - **Description**: Extracted using the regex `/<p class="paragraph-8">(.*?)<\/p>/`.
     - **Speakers**: Extracted using the regex `/<div class="session-speaker">(.*?)<\/div>/g` to match multiple speakers.
     - **Location**: Extracted using the regex `/<div class="session-location">\s*<a[^>]*>(.*?)<\/a>/`.


In [ ]:
  %%javascript
    const dateMatch = section.match(/data-w-tab="([^"]+)"/);
    if (dateMatch) {
      console.log(`Section ${index + 1} Matched Date:`, dateMatch[1]);
      // console.log(`Section Content Preview:`, section.substring(0, 500)); // Log the surrounding context
    } else {
      console.log(`No date found in Section ${index + 1}`);
    }
    const currentDate = dateMatch ? dateMatch[1].replace("th", ", 2024") : "Unknown";
    // console.log(`Processing Section ${index + 1}: Tab Name - ${currentDate}`);

    // Regex to extract tabs with their `data-w-tab` attribute and sessions
    const sessionRegex = /<div class="speaker-session-wrapper">([\s\S]*?)(?=<div class="speaker-session-wrapper"|<\/div><\/div><\/div><\/div>)/g;
    const sessionMatches = section.match(sessionRegex);

6. **Storing Extracted Data:**
   - If a session contains at least a title or start time, it is added to the `sessions` array.
   - Each session object contains the following fields:
     - `date`: The date of the session.
     - `time_start`: The start time of the session.
     - `time_end`: The end time of the session.
     - `track`: The track/category of the session.
     - `title`: The title of the session.
     - `speakers`: A string containing the names of the session's speakers.
     - `description`: A brief description of the session.
     - `location`: The location of the session.

In [ ]:
%%javascript
    if (sessionMatches) {
      sessionMatches.forEach(sessionHtml => {
        // Field regexes
        const timeStartRegex = /<div class="text-block-230">(.*?)<\/div>/;
        const timeEndRegex = /<div class="text-block-230 end">(.*?)<\/div>/;
        const trackRegex = /<div[^>]*class="session-track"[^>]*>(.*?)<\/div>/;
        const titleRegex = /<h1 class="session-ttitle">(.*?)<\/h1>/;
        const descriptionRegex = /<p class="paragraph-8">(.*?)<\/p>/;
        const speakerRegex = /<div class="session-speaker">(.*?)<\/div>/g;
        const locationRegex = /<div class="session-location">\s*<a[^>]*>(.*?)<\/a>/;

        // Extract individual fields
        const timeStartRaw = extractField(sessionHtml, timeStartRegex);
        const timeStart = timeStartRaw ? timeStartRaw.replace('-', '') : null;
        const timeEnd = extractField(sessionHtml, timeEndRegex);
        const track = extractField(sessionHtml, trackRegex);
        const title = extractField(sessionHtml, titleRegex);
        const description = extractField(sessionHtml, descriptionRegex);

      // Extract multiple speakers
      const speakers = [];
      let speakerMatch;
      while ((speakerMatch = speakerRegex.exec(sessionHtml.replace('session-speaker name', 'session-speaker'))) !== null) {
        speakers.push(speakerMatch[1].trim());
        console.log(speakers)
      }
      const speakersString = speakers.join(", "); // Join speakers with a comma and space
      console.log(speakersString)

      const location = extractField(sessionHtml, locationRegex).replace('<div class="text-block-231">','').replace('</div>','');

      // Only include sessions with at least time_start and title
      if (timeStart || title) {
        sessions.push({
          date: currentDate, // Add the event day
          time_start: timeStart,
          time_end: timeEnd,
          track,
          title,
          speakers: speakersString,
          description,
          location,
          });
        }
      });
    } else {
      console.log(`No sessions found for ${tabName}`);
    }
  });

7. **Logging Extracted Data:**
   - The extracted session data is logged using `Logger.log()`, which outputs the data in JSON format for debugging and validation.

In [ ]:
  %%javascript
  // Log the extracted data
  Logger.log(JSON.stringify(sessions, null, 2));

8. **Writing Data to Google Sheets:**
   - After processing all sections and extracting session details, the function writes the session data to a Google Sheet named `"AppScript_Sessions"`.
   - If the sheet does not already exist, it is created using `insertSheet()`. The sheet is cleared, and new headers are added to the first row.
   - The session data is written to the sheet, with each row representing a session.

In [ ]:
%%javascript
  // Step 5: Write data to Google Sheets
  const sheetName = "AppScript_Sessions";
  let sheet = SpreadsheetApp.getActiveSpreadsheet().getSheetByName(sheetName);
  if (!sheet) {
    sheet = SpreadsheetApp.getActiveSpreadsheet().insertSheet(sheetName);
  }
  sheet.clear();
  sheet.appendRow(["Date","Start Time", "End Time", "Track", "Title", "Speakers", "Description", "Location", "Planning to Go?"]);
  sessions.forEach(session => {
    sheet.appendRow([
      session.date,
      session.time_start,
      session.time_end,
      session.track,
      session.title,
      session.speakers,
      session.description,
      session.location,
    ]);
  });

9. **Adding Data Validation:**
   - The function adds data validation to the "Planning to Go?" column in the sheet. Users are allowed to select either "Yes" or "No" from a dropdown in this column.


In [ ]:
  %%javascript
  // Add data validation to "Planning to Go?" column
  const validationRange = sheet.getRange(2, 9, sheet.getLastRow() - 1); // "Planning to Go?" column (I)
  const rule = SpreadsheetApp.newDataValidation()
    .requireValueInList(["Yes", "No"], true)
    .setAllowInvalid(false)
    .build();
  validationRange.setDataValidation(rule);

10. **Returning Session Data:**
    - Finally, the function returns the `sessions` array containing all the extracted and processed session details.


In [ ]:
%%javascript
  return sessions;
}

### Example of Workflow:
- **Input:** The HTML content of the summit's schedule page..
- **Process:** The HTML is parsed, sections are identified, session details are extracted, and the data is structured.
- **Output:**  The session data is written to a Google Sheet, and data validation is applied to ensure users can select "Yes" or "No" for the "Planning to Go?" column.



In [ ]:

function extractField(html, regex) {
  const match = html.match(regex);
  return match ? match[1].trim() : null;
}

# Function: `testRegexInAppScript()`

### Purpose:
The `testRegexInAppScript()` function is designed to test a regular expression that matches the `data-w-tab` attribute in a given HTML snippet. The function extracts the value of the `data-w-tab` attribute (which typically holds the tab name or date, e.g., "December 11th") and logs it to the console. If no match is found, it logs a message indicating that no match was found.

### Breakdown of the Function:

1. **HTML Snippet:**
   - The function starts by defining a string called `htmlSnippet` which contains a portion of HTML. This HTML includes several `div` elements, some of which have the `data-w-tab` attribute with values like `"December 11th"` and `"December 12th"`.
   - This snippet is a simplified example that mimics a real HTML structure you might encounter in a webpage.

2. **Regular Expression Definition:**
   - The function defines a regular expression (`tabNameRegex`) designed to match the value of the `data-w-tab` attribute. The regular expression is:
     ```javascript
     const tabNameRegex = /data-w-tab="([^"]+)"/;
     ```
     - **`data-w-tab="([^"]+)"`**:
       - `data-w-tab="`: Matches the literal string `data-w-tab="`.
       - `([^"]+)`: A capturing group that matches one or more characters that are not a quote (`"`), which corresponds to the value of the `data-w-tab` attribute (e.g., "December 11th").
       - `"`: Matches the closing quote of the `data-w-tab` attribute value.

3. **Matching the Regular Expression:**
   - The function then applies the regular expression to the `htmlSnippet` using the `match()` method:
     ```javascript
     const match = htmlSnippet.match(tabNameRegex);
     ```
   - The `match()` method searches the `htmlSnippet` string for any occurrences of the `data-w-tab` attribute and returns an array if a match is found. The first element of this array contains the entire matched string, and the second element (accessed by `match[1]`) contains the value of the `data-w-tab` attribute.

4. **Logging the Extracted Value:**
   - If the `match()` method returns a valid match, the function logs the extracted value (the `data-w-tab` attribute value) to the console:
     ```javascript
     Logger.log(`Extracted tab name: ${match[1]}`);
     ```
   - If no match is found, the function logs the message "No match found." to indicate that the regular expression did not find any `data-w-tab` attribute in the HTML snippet.

### Example of Workflow:
- **Input:** The `htmlSnippet` contains the HTML string with multiple `div` elements, including those with `data-w-tab="December 11th"` and `data-w-tab="December 12th"`.
- **Process:** The regular expression is applied to the HTML snippet, and the `data-w-tab` attribute values are extracted.
- **Output:** The function logs the extracted tab name (e.g., `"December 11th"`) if a match is found, or it logs "No match found." if there is no match.

### Expected Output:
For the given `htmlSnippet`, the expected output would be:


In [ ]:
%%javascript
function testRegexInAppScript() {
  const htmlSnippet = '<div class="w-tab-content"><div class="w-tab-pane w--tab-active" data-w-tab="December 11th" id="Tab-1"><div class="december-11-wrapper"><div class="collection-list-wrapper-2 w-dyn-list"><div class="w-tab-pane" data-w-tab="December 12th"><div class="december-12th-wrapper" id="D">';

  // Regex to match the data-w-tab attribute
  const tabNameRegex = /data-w-tab="([^"]+)"/;

  // Extract the match
  const match = htmlSnippet.match(tabNameRegex);
  if (match) {
    Logger.log(`Extracted tab name: ${match[1]}`);
  } else {
    Logger.log("No match found.");
  }
}
